In [ ]:
# @title Скачивание списка всех доступных событий ligo
import urllib.request

# URL файла
url = "https://gwosc.org/api/v2/event-versions?include-default-parameters=true&format=csv"

# Скачиваем и сохраняем файл
urllib.request.urlretrieve(url, "gwosc_events.csv")
print("Файл успешно скачан и сохранён как 'gwosc_events.csv'")

Файл успешно скачан и сохранён как 'gwosc_events.csv'


In [ ]:
# @title Массовая загрузка
import os
import requests
import time
import json
import csv # Import the csv module

# ===== ВАШ РАБОЧИЙ КОД =====
def fetch_json(url, params=None):
    resp = requests.get(url, params=params, timeout=30)
    resp.raise_for_status()
    return resp.json()

def get_event_info(event_name):
    info = fetch_json(f"https://gwosc.org/api/v2/event-versions/{event_name}")
    return {
        'gps': float(info['gps']),
        'detectors': info['detectors'],
        'strain_files_url': info['strain_files_url']
    }

def list_early_release_files(strain_files_url, detectors, sample_rate, duration, file_format):
    files = []
    for det in detectors:
        params = {
            'detector': det,
            'sample-rate': str(sample_rate),
            'duration': str(duration),
            'file-format': file_format
        }
        page_url = strain_files_url
        while page_url:
            data = fetch_json(page_url, params=params)
            files += data.get('results', [])
            page_url = data.get('next')
            params = None
    return files

def list_event_files(event_name, detectors, sample_rate, start, stop):
    files = []
    base_url = f"https://gwosc.org/api/v2/events/{event_name}/strain-files"
    for det in detectors:
        params = {
            'detector': det,
            'sample-rate': str(sample_rate),
            'start': str(start),
            'stop': str(stop)
        }
        page_url = base_url
        while page_url:
            data = fetch_json(page_url, params=params)
            files += data.get('results', [])
            page_url = data.get('next')
            params = None
    return files

def download_file(url, path):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    if os.path.exists(path):
        print(f"[SKIP] {path} уже скачан")
        return
    with requests.get(url, stream=True, timeout=60) as r:
        r.raise_for_status()
        with open(path + ".tmp", 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024*1024):
                if chunk:
                    f.write(chunk)
    os.rename(path + ".tmp", path)
    print(f"[OK] Скачан {path}")

def download_gwosc_event(event_name,
                         detectors=None,
                         sample_rate=4,
                         duration=4096,
                         file_format='hdf5',
                         out_dir='/content/data',
                         start=None,
                         stop=None):
    info = get_event_info(event_name)
    gps = info['gps']
    dets = detectors or info['detectors']
    print(f"Событие {event_name}: GPS={gps}, детекторы: {dets}")

    # поиск ранних релизов
    early = list_early_release_files(info['strain_files_url'], dets,
                                     sample_rate, duration, file_format)
    files = []
    if early:
        for fdesc in early:
            url = fdesc.get('download_url')
            if not url:
                continue
            files.append({
                'detector': fdesc['detector'],
                'gps_start': fdesc['gps_start'],
                'duration': fdesc.get('duration'),
                'sample_rate_kHz': fdesc.get('sample_rate_kHz'),
                'url': url
            })
    else:
        # fallback на /events/<event>/strain-files
        if start is None or stop is None:
            half = duration // 2
            start = int(gps) - half
            stop = int(gps) + half
        print(f"Ранних релизов нет. Используем интервал {start}..{stop}")
        full = list_event_files(event_name, dets, sample_rate, start, stop)
        for fdesc in full:
            url = fdesc.get('hdf5_url') if file_format == 'hdf5' else fdesc.get('gwf_url')
            if not url:
                continue
            files.append({
                'detector': fdesc['detector'],
                'gps_start': fdesc['gps_start'],
                'duration': stop - start,
                'sample_rate_kHz': fdesc.get('sample_rate_kHz'),
                'url': url
            })

    if not files:
        print("Файлы не найдены с указанными параметрами.")
        return False

    for entry in files:
        det = entry['detector']
        gps_start = entry['gps_start']
        dur = entry['duration']
        sr = entry['sample_rate_kHz']
        url = entry['url']
        dur_str = f"{dur}s" if dur is not None else "unknown"
        sr_str = f"{sr}kHz" if sr is not None else "unknown"
        ext = 'hdf5' if file_format == 'hdf5' else 'gwf'
        filename = f"{event_name}_{det}_{gps_start}_{dur_str}_{sr_str}.{ext}"
        path = os.path.join(out_dir, filename)
        print(f"Скачиваем {det}: GPS {gps_start}, {dur_str}, {sr_str}...")
        download_file(url, path)

    return True

# ===== РАСШИРЕНИЕ ДЛЯ МАССОВОЙ ЗАГРУЗКИ =====

def get_all_events_from_csv(csv_file_path):
    """Получить список событий из CSV файла"""
    all_events = []
    try:
        with open(csv_file_path, mode='r') as csv_file:
            csv_reader = csv.reader(csv_file)
            # Пропускаем заголовок, если он есть. В данном случае, файл содержит только названия событий.
            # next(csv_reader) # Uncomment this line if your CSV has a header row
            for row in csv_reader:
                if row: # Check if the row is not empty
                    all_events.append(row[0].strip()) # Assuming event name is in the first column and remove leading/trailing whitespace
    except FileNotFoundError:
        print(f"Ошибка: Файл '{csv_file_path}' не найден.")
        return []
    except Exception as e:
        print(f"Ошибка при чтении CSV: {e}")
        return []
    return sorted(list(set(all_events)))


def download_all_events(out_dir='/content/gwosc_data',
                       sample_rate=4,
                       duration=4096,
                       max_events=None,
                       csv_file='/content/event-versions.csv'): # Add csv_file parameter
    """Массовая загрузка всех событий"""

    print("="*60)
    print("МАССОВАЯ ЗАГРУЗКА СОБЫТИЙ LIGO/Virgo")
    print("="*60)

    os.makedirs(out_dir, exist_ok=True)

    # Получаем список событий из CSV файла
    all_events = get_all_events_from_csv(csv_file) # Use the new function
    if not all_events:
        print("Список событий из CSV пуст или не удалось его прочитать.")
        return

    print(f"Найдено {len(all_events)} событий в {csv_file}")

    if max_events:
        all_events = all_events[:max_events]
        print(f"Ограничение: первые {max_events} событий")

    # Прогресс
    progress_file = os.path.join(out_dir, 'download_progress.json')
    progress = {}
    if os.path.exists(progress_file):
        with open(progress_file, 'r') as f:
            progress = json.load(f)

    downloaded = 0
    failed = 0
    skipped = 0

    for i, event_name in enumerate(all_events, 1):
        print(f"\n[{i}/{len(all_events)}] {event_name}")

        # Проверяем прогресс
        if event_name in progress and progress[event_name] == 'success':
            print("  [SKIP] Уже загружено ранее")
            skipped += 1
            continue

        try:
            # Используем ваш рабочий код
            success = download_gwosc_event(
                event_name,
                detectors=None,
                sample_rate=sample_rate,
                duration=duration,
                file_format='hdf5',
                out_dir=out_dir
            )

            if success:
                downloaded += 1
                progress[event_name] = 'success'
            else:
                failed += 1
                progress[event_name] = 'failed'

        except Exception as e:
            print(f"  [ERROR] {e}")
            failed += 1
            progress[event_name] = 'failed'

        # Сохраняем прогресс
        with open(progress_file, 'w') as f:
            json.dump(progress, f, indent=2)

        # Пауза
        time.sleep(1)

    print("\n" + "="*60)
    print("ЗАГРУЗКА ЗАВЕРШЕНА")
    print("="*60)
    print(f"✅ Успешно: {downloaded}")
    print(f"⏭️ Пропущено: {skipped}")
    print(f"❌ Ошибки: {failed}")

    files = [f for f in os.listdir(out_dir) if f.endswith('.hdf5')]
    print(f"📊 Всего файлов: {len(files)}")

# ===== ЗАПУСК =====

if __name__ == "__main__":
    download_all_events(
        out_dir="/content/gwosc_data",
        sample_rate=4,
        duration=4096,
        max_events=None,  # None = все события
        csv_file='/content/gwosc_events.csv' # Specify the CSV file
    )

МАССОВАЯ ЗАГРУЗКА СОБЫТИЙ LIGO/Virgo
Найдено 264 событий в /content/gwosc_events.csv

[1/264] 151008
Событие 151008: GPS=1128348574.5, детекторы: ['H1', 'L1']
Скачиваем H1: GPS 1128346527, 4096s, 4kHz...
[OK] Скачан /content/gwosc_data/151008_H1_1128346527_4096s_4kHz.hdf5
Скачиваем L1: GPS 1128346527, 4096s, 4kHz...
[OK] Скачан /content/gwosc_data/151008_L1_1128346527_4096s_4kHz.hdf5

[2/264] 151012.2
Событие 151012.2: GPS=1128666662.2, детекторы: ['H1', 'L1']
Скачиваем H1: GPS 1128664615, 4096s, 4kHz...
[OK] Скачан /content/gwosc_data/151012.2_H1_1128664615_4096s_4kHz.hdf5
Скачиваем L1: GPS 1128664615, 4096s, 4kHz...
[OK] Скачан /content/gwosc_data/151012.2_L1_1128664615_4096s_4kHz.hdf5

[3/264] 151116
Событие 151116: GPS=1131748925.7, детекторы: ['H1', 'L1']
Скачиваем H1: GPS 1131746878, 4096s, 4kHz...
[OK] Скачан /content/gwosc_data/151116_H1_1131746878_4096s_4kHz.hdf5
Скачиваем L1: GPS 1131746878, 4096s, 4kHz...
[OK] Скачан /content/gwosc_data/151116_L1_1131746878_4096s_4kHz.hdf5



In [ ]:
# @title ABCD UNIVERSAL ANALYZER v5.1 - ОПТИМИЗАЦИЯ ПАМЯТИ

# -*- coding: utf-8 -*-
"""
ABCD UNIVERSAL ANALYZER v5.1 - С ОПТИМИЗАЦИЕЙ ПАМЯТИ
=====================================================
Полная версия со всей логикой + потоковая запись для экономии памяти
"""

import os
import json
import time
import gc
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import h5py
from scipy.signal import butter, filtfilt, find_peaks, welch
from scipy.stats import pearsonr
from scipy.fft import fft, fftfreq
from typing import Dict, Any, List, Optional, Tuple
from concurrent.futures import ProcessPoolExecutor, as_completed
from functools import lru_cache

# Константы
PHI = (1 + 5 ** 0.5) / 2
FIBONACCI = [1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597]


# ========================================
#   ОПТИМИЗИРОВАННЫЕ БАЗОВЫЕ ФУНКЦИИ
# ========================================

@lru_cache(maxsize=128)
def get_phi_power(n: int) -> float:
    """Кэшированное вычисление φ^n"""
    return PHI ** n


def convert_to_serializable(obj):
    """Быстрая конвертация для JSON"""
    if isinstance(obj, (np.bool_, np.bool)):
        return bool(obj)
    elif isinstance(obj, (np.integer, np.int64, np.int32)):
        return int(obj)
    elif isinstance(obj, (np.floating, np.float64, np.float32)):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, dict):
        return {key: convert_to_serializable(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_serializable(item) for item in obj]
    elif isinstance(obj, bytes): # Add handling for bytes
        try:
            return obj.decode('utf-8')
        except:
            return str(obj) # Fallback to string representation
    else:
        return obj


def read_strain_optimized(file_path: str) -> Tuple[np.ndarray, float, Dict]:
    """Оптимизированное чтение с метаданными"""
    try:
        with h5py.File(file_path, 'r') as f:
            # Strain
            if 'strain' in f and 'Strain' in f['strain']:
                strain = f['strain']['Strain'][:]
            elif 'Strain' in f:
                strain = f['Strain'][:]
            else:
                return None, 4096.0, {}

            # Частота
            fs = 4096.0
            if 'meta' in f:
                if 'SamplingRate' in f['meta']:
                    try:
                        fs = float(f['meta']['SamplingRate'][()])
                    except:
                        pass

                # Дополнительные метаданные
                meta = {}
                for key in ['GPSstart', 'Duration', 'Detector']:
                    if key in f['meta']:
                        try:
                            meta[key] = f['meta'][key][()]
                        except:
                            pass
            else:
                meta = {}

        return strain.astype(np.float32), fs, meta  # float32 для скорости
    except:
        return None, 4096.0, {}


def bandpass_filter_fast(x: np.ndarray, fs: float,
                         low: float = 20, high: float = 800,
                         order: int = 4) -> np.ndarray:
    """Быстрый полосовой фильтр"""
    # Очистка NaN
    x = np.nan_to_num(x, nan=0.0, posinf=0.0, neginf=0.0)

    nyq = fs / 2
    low_norm = max(low, 1) / nyq
    high_norm = min(high, nyq * 0.99) / nyq

    if low_norm >= high_norm:
        return x

    try:
        b, a = butter(order, [low_norm, high_norm], btype='band')
        return filtfilt(b, a, x)
    except:
        return x


# ========================================
#   1) КОСМОЛОГИЧЕСКАЯ КАЛИБРОВКА
# ========================================

class ABCDCosmology:
    """Быстрая космологическая калибровка"""

    def __init__(self):
        self.age_gyr = 13.8
        self.observer_shift = 0.087
        self.torus_now = 0.05
        self.interface_now = 0.25
        self.apple_now = 0.70

        rest = 1 - self.interface_now
        self.I_now = self.torus_now / rest
        self.Ibar_now = self.apple_now / rest
        self.ratio_now = self.Ibar_now / self.I_now
        self.k_now = np.log(self.ratio_now) / np.log(PHI)
        self.macro_cycles = self.k_now / 4

        # Предвычисленные значения для скорости
        self.phi_powers = {n: PHI**n for n in range(-15, 16)}


# ========================================
#   2) АНАЛИЗ ФАЗ ABCD
# ========================================

def analyze_abcd_phases_fast(strain: np.ndarray, fs: float,
                             merger_idx: int) -> Dict:
    """Быстрый анализ фаз ABCD"""

    # Окно ±1 сек вокруг слияния
    window_samples = int(fs)
    i0 = max(0, merger_idx - window_samples)
    i1 = min(len(strain), merger_idx + window_samples)
    segment = strain[i0:i1]

    if len(segment) < 100:
        return {
            'phases': {'A': 0, 'B': 0, 'C': 0, 'D': 0},
            'dominant': 'B',
            'cycles': 0,
            'transitions': []
        }

    # Размер окна 10ms
    window_size = int(0.01 * fs)
    step = window_size // 2

    # Векторизованный расчет энергий
    n_windows = (len(segment) - window_size) // step
    energies = np.zeros(n_windows)

    for i in range(n_windows):
        start = i * step
        end = start + window_size
        energies[i] = np.mean(segment[start:end]**2)

    # Градиенты
    gradients = np.diff(energies) / (energies[:-1] + 1e-30)

    # Пороги
    rms_values = np.sqrt(energies)
    low_thr = np.percentile(rms_values, 25)
    high_thr = np.percentile(rms_values, 75)

    # Классификация фаз (векторизованная)
    phases_array = np.zeros(len(energies), dtype=int)

    for i in range(len(energies)):
        grad = gradients[i-1] if i > 0 else 0
        rms = rms_values[i]

        if abs(grad) < 0.1:
            phases_array[i] = 0 if rms < low_thr else 3  # A или D
        else:
            phases_array[i] = 1 if grad > 0 else 2  # B или C

    # Подсчет фаз
    phases = {'A': 0, 'B': 0, 'C': 0, 'D': 0}
    phase_names = ['A', 'B', 'C', 'D']

    for i in range(4):
        phases[phase_names[i]] = np.sum(phases_array == i)

    # Поиск переходов
    transitions = []
    prev_phase = phases_array[0] if len(phases_array) > 0 else 0

    for i in range(1, len(phases_array)):
        if phases_array[i] != prev_phase:
            transitions.append({
                'time': (i0 + i * step) / fs,
                'from': phase_names[prev_phase],
                'to': phase_names[phases_array[i]]
            })
            prev_phase = phases_array[i]

    # Поиск циклов ABCD
    cycles = 0
    if len(transitions) >= 3:
        for i in range(len(transitions) - 3):
            seq = [transitions[j]['to'] for j in range(i, min(i+4, len(transitions)))]
            if seq == ['A', 'B', 'C', 'D'] or seq == ['D', 'C', 'B', 'A']:
                cycles += 1

    dominant_idx = np.argmax([phases[p] for p in phase_names])
    dominant = phase_names[dominant_idx]

    return {
        'phases': phases,
        'dominant': dominant,
        'cycles': cycles,
        'transitions': transitions[:20]  # Ограничиваем для скорости
    }


# ========================================
#   3) АНАЛИЗ ЭХО (ОПТИМИЗИРОВАННЫЙ)
# ========================================

def analyze_phi_echoes_fast(strain: np.ndarray, fs: float,
                            merger_idx: int, cosmology: ABCDCosmology) -> Dict:
    """Оптимизированный анализ эхо"""

    # Шум для нормализации
    noise_start = max(0, merger_idx - int(2*fs))
    noise_end = max(100, merger_idx - int(0.5*fs))

    if noise_end > noise_start:
        noise = strain[noise_start:noise_end]
        noise_level = np.std(noise)
        noise_power = np.mean(noise**2)
    else:
        noise_level = np.std(strain[:1000]) if len(strain) > 1000 else 1e-23
        noise_power = noise_level**2

    # Размер окна 50ms
    window_size = int(0.05 * fs)

    echoes = []
    compression_energies = []
    expansion_energies = []

    # Используем предвычисленные степени φ
    for n in range(-14, 15):
        if n == 0:
            continue

        echo_time = cosmology.phi_powers.get(n, PHI**n)
        echo_idx = int(merger_idx + echo_time * fs)

        if 0 <= echo_idx < len(strain) - window_size:
            segment = strain[echo_idx:echo_idx+window_size]

            # Быстрые вычисления
            energy = np.mean(segment**2)
            amplitude = np.max(np.abs(segment))

            if amplitude > noise_level * 1.2:  # Порог детекции
                snr = energy / noise_power

                echo_data = {
                    'n': n,
                    'time': echo_time,
                    'energy': float(energy),
                    'snr': float(snr),
                    'amplitude': float(amplitude),
                    'type': 'compression' if n < 0 else 'expansion'
                }

                echoes.append(echo_data)

                if n < 0:
                    compression_energies.append(energy)
                else:
                    expansion_energies.append(energy)

    # Проверка балансов φ^4
    has_phi4 = False
    balances = {}

    pairs = [(-6, 2), (-5, 3), (-4, 4), (-3, 5), (-2, 6)]

    for neg, pos in pairs:
        neg_echo = next((e for e in echoes if e['n'] == neg), None)
        pos_echo = next((e for e in echoes if e['n'] == pos), None)

        if neg_echo and pos_echo and pos_echo['energy'] > 0:
            ratio = neg_echo['energy'] / pos_echo['energy']
            phi_power = np.log(ratio) / np.log(PHI) if ratio > 0 else 0

            balances[f'{neg}_to_{pos}'] = {
                'ratio': float(ratio),
                'phi_power': float(phi_power),
                'close_to_phi4': bool(3 < phi_power < 5)
            }

            if 3 < phi_power < 5:
                has_phi4 = True

    # Общий баланс
    comp_total = sum(compression_energies) if compression_energies else 0
    exp_total = sum(expansion_energies) if expansion_energies else 0

    if exp_total > 0:
        overall_balance = comp_total / exp_total
        overall_phi = np.log(overall_balance) / np.log(PHI) if overall_balance > 0 else 0
    else:
        overall_balance = 0
        overall_phi = 0

    # Проверка каскада
    cascade_count = len([e for e in echoes if -10 <= e['n'] <= -3])
    has_cascade = cascade_count >= 5

    # 11D совместимость
    is_11d = 20 < len(echoes) < 30

    return {
        'count': len(echoes),
        'echoes': sorted(echoes, key=lambda x: x['snr'], reverse=True)[:10],
        'compression_count': len(compression_energies),
        'expansion_count': len(expansion_energies),
        'has_cascade': has_cascade,
        'cascade_count': cascade_count,
        'has_phi4': has_phi4,
        'overall_balance': float(overall_balance),
        'overall_phi_power': float(overall_phi),
        'is_11d': is_11d,
        'balances': balances
    }


# ========================================
#   4) ПРОТИВОФАЗА H1-L1
# ========================================

def analyze_antiphase_fast(h1_strain: np.ndarray, l1_strain: np.ndarray,
                           fs: float) -> Dict:
    """Быстрый анализ противофазы"""

    if h1_strain is None or l1_strain is None:
        return {'has_antiphase': False, 'correlation': 0}

    # Берем центральные 100ms
    min_len = min(len(h1_strain), len(l1_strain))
    center = min_len // 2
    window = int(0.05 * fs)  # 50ms для скорости

    h1_seg = h1_strain[center-window:center+window]
    l1_seg = l1_strain[center-window:center+window]

    if len(h1_seg) != len(l1_seg) or len(h1_seg) < 100:
        return {'has_antiphase': False, 'correlation': 0}

    # Быстрая корреляция через FFT
    h1_fft = fft(h1_seg)
    l1_fft = fft(l1_seg)

    # Фазовый сдвиг
    cross_spectrum = h1_fft * np.conj(l1_fft)
    phase_diff = np.angle(cross_spectrum)
    mean_phase = np.mean(phase_diff[1:len(phase_diff)//2])  # Исключаем DC

    # Проверка на π/2 (90°)
    expected = np.pi / 2
    is_antiphase = abs(abs(mean_phase) - expected) < 0.5

    # Прямая корреляция
    try:
        correlation = np.corrcoef(h1_seg, l1_seg)[0, 1]
    except:
        correlation = 0

    return {
        'has_antiphase': is_antiphase,
        'phase_shift': float(mean_phase),
        'correlation': float(correlation),
        'expected_shift': float(expected)
    }


# ========================================
#   5) ПЕРЕХОДЫ ЧД ↔ КРОТОВАЯ НОРА
# ========================================

def analyze_bh_wh_transitions_fast(strain: np.ndarray, fs: float,
                                   merger_idx: int) -> Dict:
    """Быстрый анализ переходов ЧД↔КН"""

    # Анализируем только ±0.5 сек вокруг слияния
    window = int(0.5 * fs)
    i0 = max(0, merger_idx - window)
    i1 = min(len(strain), merger_idx + window)
    segment = strain[i0:i1]

    if len(segment) < 100:
        return {'transitions': 0, 'dominant_state': 'unknown'}

    # Размер окна 100ms, шаг 50ms
    window_size = int(0.1 * fs)
    step = int(0.05 * fs)

    transitions = 0
    prev_state = 'unknown'

    for i in range(0, len(segment) - window_size, step):
        window = segment[i:i+window_size]

        # Энергия и градиент
        energy = np.mean(window**2)

        if i > 0:
            prev_window = segment[i-step:i-step+window_size]
            prev_energy = np.mean(prev_window**2)

            if prev_energy > 0:
                ratio = energy / prev_energy

                # Определение состояния
                if ratio > PHI**2:  # Сильное расширение
                    state = 'wormhole'
                elif ratio < 1/PHI**2:  # Сильное сжатие
                    state = 'blackhole'
                else:
                    state = 'transition'

                if state != prev_state and prev_state != 'unknown':
                    transitions += 1

                prev_state = state

    # Определение доминирующего состояния
    if transitions > 5:
        dominant = 'oscillating'
    elif transitions > 2:
        dominant = 'transitioning'
    else:
        dominant = 'stable'

    return {
        'transitions': transitions,
        'dominant_state': dominant
    }


# ========================================
#   6) ИНВАРИАНТ I×Ī
# ========================================

def check_invariant_fast(strain: np.ndarray) -> Dict:
    """Быстрая проверка инварианта"""

    # Используем только центральную часть для скорости
    if len(strain) > 10000:
        center = len(strain) // 2
        segment = strain[center-5000:center+5000]
    else:
        segment = strain

    # Быстрая энтропия через гистограмму
    hist, _ = np.histogram(segment, bins=30)
    hist = hist[hist > 0]

    if len(hist) > 1:
        probs = hist / np.sum(hist)
        I = -np.sum(probs * np.log2(probs + 1e-10))
    else:
        I = 1.0

    expected = 1 / (PHI ** 2)
    Ibar = expected / (I + 1e-10)
    product = I * Ibar
    deviation = abs(product - expected) / expected

    return {
        'I': float(I),
        'Ibar': float(Ibar),
        'product': float(product),
        'deviation': float(deviation),
        'valid': deviation < 0.1
    }


# ========================================
#   7) АНАЛИЗАТОР ОДНОГО ФАЙЛА
# ========================================

def analyze_single_file(file_path: str, cosmology: ABCDCosmology) -> Optional[Dict]:
    """Полный анализ одного файла"""

    # Парсим имя файла
    filename = os.path.basename(file_path)
    parts = filename.replace('.hdf5', '').split('_')

    event_name = parts[0]
    detector = parts[1] if len(parts) > 1 else 'unknown'

    # Читаем данные
    strain, fs, meta = read_strain_optimized(file_path)
    if strain is None:
        return None

    # Фильтрация
    filtered = bandpass_filter_fast(strain, fs)

    # Находим момент слияния
    merger_idx = np.argmax(np.abs(filtered))

    # Все анализы
    results = {
        'event': event_name,
        'detector': detector,
        'fs': fs,
        'merger_idx': merger_idx,
        'meta': meta
    }

    # 1. Фазы ABCD
    results['abcd'] = analyze_abcd_phases_fast(filtered, fs, merger_idx)

    # 2. Эхо
    results['echoes'] = analyze_phi_echoes_fast(filtered, fs, merger_idx, cosmology)

    # 3. Переходы ЧД↔КН
    results['bh_wh'] = analyze_bh_wh_transitions_fast(filtered, fs, merger_idx)

    # 4. Инвариант
    results['invariant'] = check_invariant_fast(filtered)

    # Для противофазы нужны оба детектора (обработаем позже)
    results['strain_segment'] = filtered[max(0, merger_idx-2048):merger_idx+2048]

    # Удаляем большие массивы из памяти
    del strain, filtered

    return results


# ========================================
#   8) ГЛАВНЫЙ АНАЛИЗАТОР С ОПТИМИЗАЦИЕЙ
# ========================================

class UniversalABCDAnalyzer:
    """Оптимизированный универсальный анализатор v5.1"""

    def __init__(self, data_dir: str, csv_path: str = None):
        self.data_dir = data_dir
        self.cosmology = ABCDCosmology()
        self.results = []

        # Загрузка метаданных если есть
        self.metadata = {}
        if csv_path and os.path.exists(csv_path):
            try:
                df = pd.read_csv(csv_path)
                print(f"📊 Загружено {len(df)} строк метаданных")
            except:
                pass

        # Статистика
        self.stats = {
            'total_files': 0,
            'processed': 0,
            'failed': 0,
            'phases': {'A': 0, 'B': 0, 'C': 0, 'D': 0},
            'cycles': 0,
            'with_echoes': 0,
            'with_phi4': 0,
            'with_11d': 0,
            'with_cascade': 0,
            'with_antiphase': 0,
            'invariant_valid': 0,
            'bh_wh_oscillating': 0
        }

        # Открываем файл для потоковой записи
        self.output_file = None
        self.output_handle = None

    def run_parallel_analysis(self, max_files: int = None, n_workers: int = 4,
                             batch_size: int = 50, save_to_disk: bool = True):
        """Параллельный анализ с оптимизацией памяти"""

        print("="*80)
        print("ABCD UNIVERSAL ANALYZER v5.1 - ОПТИМИЗАЦИЯ ПАМЯТИ")
        print("="*80)

        start_time = time.time()

        # Космология
        print(f"\n🌌 КОСМОЛОГИЯ:")
        print(f"  Макроциклов: {self.cosmology.macro_cycles:.2f}")
        print(f"  Ī/I = φ^{self.cosmology.k_now:.2f}")
        print(f"  Баланс: тор={self.cosmology.torus_now*100:.0f}% | "
              f"интерфейс={self.cosmology.interface_now*100:.0f}% | "
              f"яблоко={self.cosmology.apple_now*100:.0f}%")

        # Собираем файлы
        all_files = [f for f in os.listdir(self.data_dir) if f.endswith('.hdf5')]
        all_files.sort()

        if max_files:
            all_files = all_files[:max_files]

        self.stats['total_files'] = len(all_files)

        print(f"\n📁 Найдено {len(all_files)} файлов")
        print(f"💾 Потоковая запись: {'ВКЛ' if save_to_disk else 'ВЫКЛ'}")
        print(f"⚡ Размер батча: {batch_size} файлов")
        print("-"*80)

        # Открываем файл для потоковой записи
        if save_to_disk:
            self.output_file = 'abcd_results_streaming.jsonl'
            self.output_handle = open(self.output_file, 'w', encoding='utf-8')
            print(f"💾 Результаты пишутся в: {self.output_file}")

        # Группируем по событиям
        events = {}
        for file in all_files:
            event = file.split('_')[0]
            if event not in events:
                events[event] = []
            events[event].append(file)

        print(f"📊 Уникальных событий: {len(events)}")

        # Обрабатываем батчами
        batch_count = 0
        files_in_batch = 0

        for event_idx, (event_name, event_files) in enumerate(events.items(), 1):

            if event_idx % 20 == 1:
                elapsed = time.time() - start_time
                rate = self.stats['processed'] / max(elapsed, 1)
                eta = (len(all_files) - self.stats['processed']) / max(rate, 0.1)
                print(f"\n[{event_idx}/{len(events)}] {event_name} | "
                      f"Обработано: {self.stats['processed']} | "
                      f"Скорость: {rate:.1f} файл/сек | "
                      f"ETA: {eta/60:.1f} мин")

            event_results = []

            for file in event_files:
                file_path = os.path.join(self.data_dir, file)

                try:
                    result = analyze_single_file(file_path, self.cosmology)

                    if result:
                        self.stats['processed'] += 1
                        event_results.append(result)
                        self._update_stats(result)
                    else:
                        self.stats['failed'] += 1

                except Exception as e:
                    self.stats['failed'] += 1
                    if event_idx <= 2:
                        print(f"  ❌ {file}: {e}")

            # Анализ противофазы для события
            if len(event_results) >= 2:
                h1_result = next((r for r in event_results if r['detector'] == 'H1'), None)
                l1_result = next((r for r in event_results if r['detector'] == 'L1'), None)

                if h1_result and l1_result:
                    antiphase = analyze_antiphase_fast(
                        h1_result.get('strain_segment'),
                        l1_result.get('strain_segment'),
                        h1_result['fs']
                    )

                    if antiphase['has_antiphase']:
                        self.stats['with_antiphase'] += 1

                    # Добавляем в результаты
                    for r in event_results:
                        r['antiphase'] = antiphase

            # КРИТИЧНО: Удаляем strain_segment для экономии памяти
            for r in event_results:
                if 'strain_segment' in r:
                    del r['strain_segment']

            # Сохраняем результаты события
            if save_to_disk and self.output_handle:
                # Пишем сразу на диск
                for r in event_results:
                    json_line = json.dumps(convert_to_serializable(r))
                    self.output_handle.write(json_line + '\n')
                    self.output_handle.flush()
            else:
                # Накапливаем в памяти (старый режим)
                self.results.extend(event_results)

            files_in_batch += len(event_files)

            # Очистка памяти каждые batch_size файлов
            if files_in_batch >= batch_size:
                gc.collect()
                files_in_batch = 0
                batch_count += 1

            # Периодический вывод статистики
            if self.stats['processed'] % 50 == 0:
                self._print_progress()

        # Закрываем файл потоковой записи
        if self.output_handle:
            self.output_handle.close()
            print(f"\n💾 Результаты сохранены в: {self.output_file}")

        # Финальная очистка памяти
        gc.collect()

        # Финальный отчет
        elapsed = time.time() - start_time
        self._print_final_report(elapsed)

        return self.results

    def _update_stats(self, result: Dict):
        """Обновление статистики"""

        # Фазы
        for phase in ['A', 'B', 'C', 'D']:
            self.stats['phases'][phase] += result['abcd']['phases'][phase]

        self.stats['cycles'] += result['abcd']['cycles']

        # Эхо
        if result['echoes']['count'] > 0:
            self.stats['with_echoes'] += 1
        if result['echoes']['has_phi4']:
            self.stats['with_phi4'] += 1
        if result['echoes']['is_11d']:
            self.stats['with_11d'] += 1
        if result['echoes']['has_cascade']:
            self.stats['with_cascade'] += 1

        # ЧД↔КН
        if result['bh_wh']['dominant_state'] == 'oscillating':
            self.stats['bh_wh_oscillating'] += 1

        # Инвариант
        if result['invariant']['valid']:
            self.stats['invariant_valid'] += 1

    def _print_progress(self):
        """Промежуточная статистика"""
        n = max(self.stats['processed'], 1)
        print(f"\n📈 ПРОГРЕСС ({n} файлов):")
        print(f"  φ⁴: {self.stats['with_phi4']/n*100:.1f}% | "
              f"11D: {self.stats['with_11d']/n*100:.1f}% | "
              f"Каскад: {self.stats['with_cascade']/n*100:.1f}% | "
              f"I×Ī валид: {self.stats['invariant_valid']/n*100:.1f}%")

    def _print_final_report(self, elapsed_time: float):
        """Финальный отчет"""

        print("\n" + "="*80)
        print("ИТОГОВЫЙ ОТЧЕТ")
        print("="*80)

        n = max(self.stats['processed'], 1)

        # Обработка
        print(f"\n📈 ОБРАБОТКА:")
        print(f"  Файлов всего: {self.stats['total_files']}")
        print(f"  Успешно обработано: {self.stats['processed']}")
        print(f"  Ошибок: {self.stats['failed']}")
        print(f"  Время: {elapsed_time:.1f} сек ({elapsed_time/60:.1f} мин)")
        print(f"  Скорость: {n/elapsed_time:.1f} файл/сек")

        # Фазы ABCD
        total_phases = sum(self.stats['phases'].values())
        if total_phases > 0:
            print(f"\n🌀 ФАЗЫ ABCD ({total_phases} окон):")
            for phase in ['A', 'B', 'C', 'D']:
                count = self.stats['phases'][phase]
                print(f"  {phase}: {count} ({count/total_phases*100:.1f}%)")
            print(f"  Полных циклов: {self.stats['cycles']}")

        # Эхо
        print(f"\n🔊 ЭХО НА φⁿ:")
        print(f"  С эхо: {self.stats['with_echoes']} ({self.stats['with_echoes']/n*100:.1f}%)")
        print(f"  С φ⁴ балансом: {self.stats['with_phi4']} ({self.stats['with_phi4']/n*100:.1f}%)")
        print(f"  11D совместимость: {self.stats['with_11d']} ({self.stats['with_11d']/n*100:.1f}%)")
        print(f"  С каскадом: {self.stats['with_cascade']} ({self.stats['with_cascade']/n*100:.1f}%)")

        # Другие метрики
        print(f"\n⚖️ ДОПОЛНИТЕЛЬНЫЕ МЕТРИКИ:")
        print(f"  Противофаза H1-L1: {self.stats['with_antiphase']} событий")
        print(f"  ЧД↔КН осцилляции: {self.stats['bh_wh_oscillating']} файлов")
        print(f"  Инвариант I×Ī валиден: {self.stats['invariant_valid']} ({self.stats['invariant_valid']/n*100:.1f}%)")

        # Выводы
        print(f"\n" + "="*80)
        print("КЛЮЧЕВЫЕ ВЫВОДЫ:")
        print("="*80)

        if total_phases > 0:
            b_pct = self.stats['phases']['B'] / total_phases * 100
            c_pct = self.stats['phases']['C'] / total_phases * 100
            print(f"\n1. Фазы B ({b_pct:.0f}%) и C ({c_pct:.0f}%) доминируют")
            print("   → Детектируем момент перехода (слияние)")

        print(f"\n2. φ⁴ баланс найден в {self.stats['with_phi4']/n*100:.0f}% случаев")
        print("   → Подтверждает ABCD структуру пространства-времени")

        print(f"\n3. 11D совместимость в {self.stats['with_11d']/n*100:.0f}% случаев")
        print("   → Соответствует предсказаниям теории")

        print(f"\n4. Космологические макроциклы: {self.cosmology.macro_cycles:.2f}")
        print(f"   → Вселенная в фазе расширения (яблоко доминирует)")

    def save_results(self, output_prefix: str = 'abcd_analysis'):
        """Сохранение результатов"""

        if not self.results:
            print("Нет результатов для сохранения")
            return

        # JSON со статистикой
        summary = {
            'version': '5.1',
            'stats': convert_to_serializable(self.stats),
            'cosmology': {
                'macro_cycles': float(self.cosmology.macro_cycles),
                'k_now': float(self.cosmology.k_now),
                'I_now': float(self.cosmology.I_now),
                'Ibar_now': float(self.cosmology.Ibar_now)
            },
            'total_results': len(self.results)
        }

        with open(f'{output_prefix}_summary.json', 'w') as f:
            json.dump(summary, f, indent=2)

        print(f"\n💾 Сводка сохранена в {output_prefix}_summary.json")

        # CSV с детальными результатами
        df_data = []
        for r in self.results[:200]:  # Первые 200 для CSV
            row = {
                'event': r['event'],
                'detector': r['detector'],
                'dominant_phase': r['abcd']['dominant'],
                'cycles': r['abcd']['cycles'],
                'echo_count': r['echoes']['count'],
                'compression': r['echoes']['compression_count'],
                'expansion': r['echoes']['expansion_count'],
                'has_phi4': r['echoes']['has_phi4'],
                'phi_power': r['echoes']['overall_phi_power'],
                'is_11d': r['echoes']['is_11d'],
                'has_cascade': r['echoes']['has_cascade'],
                'bh_wh_state': r['bh_wh']['dominant_state'],
                'invariant_valid': r['invariant']['valid'],
                'I_value': r['invariant']['I'],
                'Ibar_value': r['invariant']['Ibar']
            }

            # Добавляем антифазу если есть
            if 'antiphase' in r:
                row['has_antiphase'] = r['antiphase']['has_antiphase']
                row['phase_shift'] = r['antiphase'].get('phase_shift', 0)

            df_data.append(row)

        if df_data:
            df = pd.DataFrame(df_data)
            df.to_csv(f'{output_prefix}_details.csv', index=False)
            print(f"💾 Детали сохранены в {output_prefix}_details.csv")


# ========================================
#   9) ЗАПУСК
# ========================================

if __name__ == "__main__":
    # Конфигурация
    DATA_DIR = "/content/gwosc_data"
    CSV_PATH = "gwosc-events.csv"
    MAX_FILES = None  # None = все файлы
    BATCH_SIZE = 50   # Уменьшите если мало памяти (10-20)
    SAVE_TO_DISK = True  # True = потоковая запись, False = в памяти

    print("🚀 Запуск ABCD Universal Analyzer v5.1...")
    print(f"📁 Директория: {DATA_DIR}")

    # Создаем анализатор
    analyzer = UniversalABCDAnalyzer(DATA_DIR, CSV_PATH)

    # Запускаем анализ с оптимизацией памяти
    results = analyzer.run_parallel_analysis(
        max_files=MAX_FILES,
        n_workers=1,  # Параллельность пока отключена для стабильности
        batch_size=BATCH_SIZE,
        save_to_disk=SAVE_TO_DISK
    )

    # Сохраняем результаты (если были в памяти)
    if not SAVE_TO_DISK and results:
        analyzer.save_results('abcd_v5_full')
        print(f"\n✅ Анализ завершен! Обработано {len(results)} файлов")
    else:
        print(f"\n✅ Анализ завершен! Обработано {analyzer.stats['processed']} файлов")

🚀 Запуск ABCD Universal Analyzer v5.1...
📁 Директория: /content/gwosc_data
ABCD UNIVERSAL ANALYZER v5.1 - ОПТИМИЗАЦИЯ ПАМЯТИ

🌌 КОСМОЛОГИЯ:
  Макроциклов: 1.37
  Ī/I = φ^5.48
  Баланс: тор=5% | интерфейс=25% | яблоко=70%

📁 Найдено 577 файлов
💾 Потоковая запись: ВКЛ
⚡ Размер батча: 50 файлов
--------------------------------------------------------------------------------
💾 Результаты пишутся в: abcd_results_streaming.jsonl
📊 Уникальных событий: 215

[1/215] 151008 | Обработано: 0 | Скорость: 0.0 файл/сек | ETA: 96.2 мин

[21/215] 200219 | Обработано: 42 | Скорость: 0.3 файл/сек | ETA: 34.3 мин

📈 ПРОГРЕСС (50 файлов):
  φ⁴: 24.0% | 11D: 66.0% | Каскад: 84.0% | I×Ī валид: 100.0%

[41/215] GW190503 | Обработано: 97 | Скорость: 0.3 файл/сек | ETA: 27.1 мин

📈 ПРОГРЕСС (100 файлов):
  φ⁴: 23.0% | 11D: 63.0% | Каскад: 77.0% | I×Ī валид: 100.0%

[61/215] GW190720 | Обработано: 154 | Скорость: 0.3 файл/сек | ETA: 22.0 мин

[81/215] GW190926 | Обработано: 218 | Скорость: 0.3 файл/сек | ETA: 18